<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex09/Pedro_Cintra/Exerc%C3%ADcio_9_235648.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Pedro Henrique Pinheiro Cintra'
print(f'Meu nome é {nome}')

Meu nome é Pedro Henrique Pinheiro Cintra


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.8 MB/s 
     |████████████████████████████████| 6.6 MB 54.6 MB/s 
     |████████████████████████████████| 596 kB 49.5 MB/s 
     |████████████████████████████████| 86 kB 7.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [4]:
# Check which GPU we are using
!nvidia-smi

Wed Jun  1 23:23:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [55]:
from typing import List
from tqdm import tqdm

def tokenize(text: str, tokenizer):
    # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        # Escreva aqui seu código.        
        self.token_texts = []
    
        for text in tqdm(texts):
          token = tokenize(f'{tokenizer.cls_token} {text}', tokenizer)
          n_sep = max(len(token) // max_seq_length,1) # Numero de separações
          pad_token = [tokenizer.pad_token_id]
          token += pad_token*max(0, max_seq_length - len(token) + 1)

          # for i in tqdm(range(0, n_sep)):
          for i in range(0, n_sep):
            if i*max_seq_length + max_seq_length < len(token):
              self.token_texts.append(token[i: i + max_seq_length + 1])
              continue
            else:
              pass

            self.token_texts.append(token[-max_seq_length - 1:])
        self.token_texts = torch.LongTensor(self.token_texts)

    def __len__(self):
        # Escreva aqui seu código.
        return len(self.token_texts)

    def __getitem__(self, idx):
        # Escreva aqui seu código.
        return self.token_texts[idx][:-1], self.token_texts[idx][1:]

## Testando se a implementação do MyDataset está correta

In [52]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 2
print('\n'
      'Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('\n'
      'Passou no assert de dataset.')

100%|██████████| 1/1 [00:00<00:00, 10565.00it/s]

100%|██████████| 2/2 [00:00<00:00, 81.96it/s]


Passou no assert de tamanho do dataset.

Passou no assert de dataset.


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [53]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

File ‘sample-1gb.txt’ already there; not retrieving.



In [85]:
# Load datasets
max_seq_length = 9

train_examples = 10000
valid_examples = 2000
test_examples = 2000

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

max_lines = train_examples + valid_examples + test_examples
print(f'Truncating to {max_lines} lines.')
texts = texts[:max_lines]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

Read 250000 lines.
Truncating to 28000 lines.


100%|██████████| 4000/4000 [01:06<00:00, 60.53it/s]


In [86]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 2492844
valid examples: 502180
test examples: 469093


In [87]:
import torch.nn.functional as F

class MultiHeadSelfAttentionLayer(torch.nn.Module):
    def __init__(self, padding_idx, n_heads, dim, max_length):
        super().__init__()

        self.n_heads = n_heads
        self.dim = dim
        self.max_length = max_length
        self.padding_idx = padding_idx

        self.Wq = torch.nn.Linear(dim, dim, bias = False)
        self.Wk = torch.nn.Linear(dim, dim, bias = False)
        self.Wv = torch.nn.Linear(dim, dim, bias = False)
        self.Wo = torch.nn.Linear(dim, dim, bias = False)

        self.feed_forward = torch.nn.Sequential(
            torch.nn.Linear(dim, self.dim),
            torch.nn.ReLU(),
            torch.nn.Linear(self.dim, dim)
        )

        self.layer_norm1 = torch.nn.LayerNorm(self.dim, eps=1e-6)
        self.layer_norm2 = torch.nn.LayerNorm(self.dim, eps=1e-6)
    
    def attention(self, q, k, v, mask):
      scores = torch.matmul(q, k.transpose(-1, -2))
      scores /= math.sqrt(self.dim // self.n_heads)
      if mask is not None:
        scores += mask[:, None, :]
      else:
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.dim // self.n_heads)

      out = torch.matmul(F.softmax(scores, dim=-1), v)

      return out

    def multihead_attention(self, inputs, pad_mask):
      batch_size = inputs.shape[0]

      q = self.Wq(inputs).reshape(batch_size, self.max_length, self.n_heads, self.dim // self.n_heads)
      k = self.Wk(inputs).reshape(batch_size, self.max_length, self.n_heads, self.dim // self.n_heads)
      v = self.Wv(inputs).reshape(batch_size, self.max_length, self.n_heads, self.dim // self.n_heads)

      q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)

      output = self.attention(q, k, v, pad_mask)

      output = output.transpose(1, 2).contiguous()
      output = output.reshape(batch_size, self.max_length, self.dim)

      return self.Wo(output)

    def forward(self, batch_token_ids, pad_mask):
        x = batch_token_ids
        x = self.layer_norm1(x + self.multihead_attention(x, pad_mask))
        x = self.layer_norm2(x + self.feed_forward(x))
        return x

class LanguageModel(torch.nn.Module):
    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            pad_token_id (int): id of the pad token that will be ignored in the attention.
        """
        # Escreva seu código aqui.
        super().__init__()

        self.pad_token_id = pad_token_id
        self.embedding_layer = torch.nn.Embedding(num_embeddings=vocab_size,embedding_dim=dim,padding_idx=pad_token_id)
        self.pos_embeds = torch.nn.Linear(dim, max_seq_length, bias=False)

        self.att = torch.nn.ModuleList()

        for _ in range(n_layers):
          att = MultiHeadSelfAttentionLayer(padding_idx=pad_token_id, n_heads=8, dim=dim, max_length=max_seq_length)
          self.att.append(att)

        self.feed_forward = torch.nn.Sequential(
            torch.nn.Linear(dim, 512),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.15),
            torch.nn.Linear(512, vocab_size)
        )


    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)
            
        Returns:
            logits of shape (batch_size, max_seq_length, vocab_size)
        """
        embeddings = self.embedding_layer(inputs)
        pos_att = embeddings + self.pos_embeds.weight
        batch_size = inputs.shape[0]
        max_seq_length = inputs.shape[-1]

        pad_mask = (inputs != self.pad_token_id)
        attention_mask = torch.triu(torch.full(inputs.unsqueeze(1).repeat(1,inputs.shape[-1],1).shape, float('-inf'), device=inputs.device), diagonal=1)
        attention_mask = attention_mask.masked_fill(~pad_mask.unsqueeze(1), float('-inf'))

        out = pos_att.to(inputs.device)
        for layer in self.att:
          out = layer(out, attention_mask)
  
        out = self.feed_forward(out)

        return out

## Teste o modelo com um exemplo

In [88]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([1, 9])
sample_output.shape: torch.Size([1, 9, 29794])


In [89]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 17274914


## Assert da Perplexidade


In [90]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              30665
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [91]:
max_examples = 150_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 31299.83, valid ppl: 29869.37
10000 steps; 10240000 examples so far; train ppl: 243.45, valid ppl: 362.56
20000 steps; 20480000 examples so far; train ppl: 116.99, valid ppl: 486.00


KeyboardInterrupt: ignored

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [92]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 490.93613643719794


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [93]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz a
Eu gosto de comer pizza pois me faz a atenção
Eu gosto de comer pizza pois me faz a atenção de
Eu gosto de comer pizza pois me faz a atenção de um
Eu gosto de comer pizza pois me faz a atenção de um novo
Eu gosto de comer pizza pois me faz a atenção de um novo modelo
Eu gosto de comer pizza pois me faz a atenção de um novo modelo de
Eu gosto de comer pizza pois me faz a atenção de um novo modelo de organização
Eu gosto de comer pizza pois me faz a atenção de um novo modelo de organização,
Eu gosto de comer pizza pois me faz a atenção de um novo modelo de organização, que
Eu gosto de comer pizza pois me faz a atenção de um novo modelo de organização, que se
Eu gosto de comer pizza pois me faz a atenção de um novo modelo de organização, que se caracteriza
Eu gosto de comer pizza pois me faz a atenção de um novo modelo de organização, que se caracterizar
Eu gosto de comer pizza pois me faz a atenção de um novo modelo de organização, que se caracteriza

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.